In [ ]:
pip install langchain langchain-community langchain-text-splitters langchain-huggingface langchain-groq chromadb sentence-transformers unstructured


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.6 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=47feeb6ae1a2583aeec11fbe42ce51c8416eccdcaa139ffce0e2d160c30d1234
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [ ]:
import os


from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA


GROQ_API_KEY = "your_api_key"
PERSIST_DIR = "./chroma_db"

# ---- Hardcode the URL you want to index ----
URL = "https://en.wikipedia.org/wiki/San_Francisco"

# 1. Load and split
loader = UnstructuredURLLoader(urls=[URL])
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

# 2. Embeddings + VectorStore
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = Chroma(persist_directory=PERSIST_DIR, embedding_function=embeddings)
vectordb.add_documents(chunks)
vectordb.persist()

# 3. Retriever + LLM (Groq)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model="qwen/qwen3-32b",  # adjust model as needed
    temperature=0,
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
)

# 4. Ask questions
query = "history of san fransisco"
result = qa({"query": query})

print("\n=== ANSWER ===\n")
print(result["result"])

print("\n=== SOURCES ===\n")
for doc in result["source_documents"]:
    print(doc.metadata.get("source", "unknown"))



=== ANSWER ===

<think>
Okay, the user asked about the history of San Francisco. Let me start by recalling what I know. San Francisco is a city in California, USA. It was founded in the 18th century, right? I think the Spanish established a mission there. Maybe Mission San Francisco de Asís? That was in 1776. Then it became part of Mexico after Mexico's independence from Spain. The Mexican-American War led to the US taking over, so maybe the city became part of the US in the mid-19th century.

The Gold Rush in 1849 brought a lot of people, which made the city grow quickly. The population exploded. Then there was the 1906 earthquake and fire, which destroyed much of the city. After that, they rebuilt. The city hosted the Panama-Pacific Exposition in 1915 to celebrate the opening of the Panama Canal. That's a big event.

In the 20th century, San Francisco became known for the counterculture movement in the 1960s, like the Summer of Love. The LGBTQ+ community also found a home there, esp